# Building KRR NMR prediction model

In this tutorial, we build the kernel ridge regression model which predicts nmr values by using m3gnet descriptor. <br>
Before building model, please download dataset from figshare.

In [2]:
# !pip list

In [1]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import DotProduct

from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.fixes import loguniform
from sklearn.preprocessing import normalize

import time
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.kernel_ridge import KernelRidge

import pandas as pd
import sys
import numpy as np
import optuna
from scipy import stats
from m3gnet.models import M3GNet, M3GNetCalculator, Potential

We build $^{13}C$ nmr prediction model. We use the dataset ""

In [4]:
# Loading Pretrained model
potential = Potential(M3GNet.load())
calculator = M3GNetCalculator(potential=potential, stress_weight=0.01)

2024-03-27 01:51:34.459195: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-27 01:51:34.459247: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-27 01:51:34.459276: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5e80501d8c5c): /proc/driver/nvidia/version does not exist
2024-03-27 01:51:34.459787: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.xyz import XYZ

In [6]:
xyz = XYZ.from_file('../../data/NMR/tutorial/nicotine.xyz')
nicotine = AseAtomsAdaptor.get_atoms(xyz.all_molecules[0])

## visualize

In [7]:
from ase.visualize import view
view(nicotine, viewer="ngl")

## Generating descritptor

By executing the "get_potential_energy" method, we can get m3gnet descriptor.

In [8]:
nicotine.set_calculator(calculator)
nicotine.get_potential_energy()

-157.98018

Now, we get 'each_atom_energy.csv', in which there are 64 dimentions m3gnet descriptor vectors in each atom.

In generating descriptors for many atoms, we recommend to use the python file "m3gnet_gnn_tf_generator.py"